# **S&OP 가사의 시나리오를 변환하기**

1) 시나리오 워드 파일 --> 음성 파일

2) 음성 파일 --> 텍스트 파일

In [1]:
pip install python-docx gtts pydub SpeechRecognition

In [2]:
from docx import Document
from gtts import gTTS

# Load the document
doc_path = "/content/sample_data/S&OP녹취록.docx"
doc = Document(doc_path)

# Extract text from the document
full_text = []
for paragraph in doc.paragraphs:
    full_text.append(paragraph.text)
text = '\n'.join(full_text)

# Convert text to speech
tts = gTTS(text, lang='ko')
tts.save("/content/sample_data/S&OP녹취록_audio.mp3")

print("음성 파일 변환 완료: S&OP녹취록_audio.mp3")



음성 파일 변환 완료: S&OP녹취록_audio.mp3


In [6]:
!pip install vosk pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [7]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-ko-0.22.zip -O /content/vosk-model-small-ko-0.22.zip
!unzip /content/vosk-model-small-ko-0.22.zip -d /content/


--2025-02-14 02:03:11--  https://alphacephei.com/vosk/models/vosk-model-small-ko-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86914329 (83M) [application/zip]
Saving to: ‘/content/vosk-model-small-ko-0.22.zip’

/content/vosk-model 100%[===================>]  82.89M  22.4MB/s    in 3.7s    

2025-02-14 02:03:15 (22.4 MB/s) - ‘/content/vosk-model-small-ko-0.22.zip’ saved [86914329/86914329]

Archive:  /content/vosk-model-small-ko-0.22.zip
   creating: /content/vosk-model-small-ko-0.22/
   creating: /content/vosk-model-small-ko-0.22/am/
  inflating: /content/vosk-model-small-ko-0.22/am/final.mdl  
   creating: /content/vosk-model-small-ko-0.22/graph/
  inflating: /content/vosk-model-small-ko-0.22/graph/phones.txt  
  inflating: /content/vosk-model-small-ko-0.22/graph/disambig_tid.int  
  inflating: /content/vos

In [9]:
from vosk import Model, KaldiRecognizer
import wave
import json

# 모델 경로 설정
model_path = "/content/vosk-model-small-ko-0.22"
model = Model(model_path)

# 음성 파일 열기
wav_path = "/content/sample_data/S&OP녹취록_audio.mp3"

# MP3 -> WAV 변환 (VOSK는 WAV만 지원)
from pydub import AudioSegment
audio = AudioSegment.from_mp3(wav_path)
wav_path_converted = "/content/sample_data/S&OP녹취록_audio_converted.wav"
audio.export(wav_path_converted, format="wav")

# WAV 파일 열기
wf = wave.open(wav_path_converted, "rb")

recognizer = KaldiRecognizer(model, wf.getframerate())

results = []

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if recognizer.AcceptWaveform(data):
        result = json.loads(recognizer.Result())
        results.append(result['text'])

# 최종 텍스트 조합
full_text = ' '.join(results)

# 텍스트 파일로 저장
with open("/content/sample_data/S&OP녹취록_vosk_출처_오디오.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

print("인식된 텍스트:")
print(full_text)
print("텍스트 저장 완료: /content/sample_data/S&OP녹취록_vosk_출처_오디오.txt")



인식된 텍스트:
사업부장 만년 하세요 여러분 오늘 회의에서는 지난 [1] 월 에 실적과 각 부서에서 제시 한 개선대책 에 대해 다시 한 번 심 도 있게 논의 하고 스킬 과 부장이 제시 한 숫자 들의 근거 와 그 산촌 반칙 에 대해 자세히 설명 양 의 주실 것을 유청 하고자 합니다 우리 회사는 올해 글로벌 자동차 부품 시장 점유율 [15] % 달성 을 목표로 하고 있습니다 [1] 월 전체매출 은 백억원 중 [92] 억원 을 달성 했으나 뉴스 지 치는 고객 주문 칠서 와 나킬 지압 그리고 생산 현장에 설비 오류 및 품질 유출 등 여러 요인을 위해 [8] % 이다 가게 되었습니다 오늘 은 각 부서 별로 제시 한 수치 와 그 근거를 보다 구체적으로 설명 하며 실행 계획을 전체적으로 재선거 마 겠습니다 무산 영업 부장님 부터 시작해 출시 제어 영업부장 [4] 사 OP 장림 저희 영업팀 은 지난 [1] 월 구 [22] 억 억원 의 매출을 기록 했습니다 이 수치는 전년도 [1] 월 평균 매출 [98] 억원 대 일 약 [6] 억원 감소 한 스 치러 들킬 자동차 후엔 고객 과 부품 공급사 을 거래 에서 주문 칠 소율이 평균 사 [쩜] [삼] 퍼센트 를 기록 한 것 씬 주요 원인 입니다 예 를 들어 A 사의 경우 일원 거래일 길 이 [20] 억원 이었는데 당초 계약 대비 [5] % 이 [3] 주문 취소 가 있었으나 실제로 는 사 [쩜] [오] % 칠 소율이 발생 하여 약 [1] 억원 상당의 주문 이 출소 되었습니다 릴리스 츠 는 지난 육 개월 월간 고객 사변 주문 출소 건수를 집계한 결과 이며 레이 사건 의 평균 칠 소율이 사 [쩜] [칠] % 에서 최근 사 [쩜] [오] 퍼센트 로 약간 하락한 수치를 보 였습니다 B 사 에서는 [15] 억 원 중 영 [쩜] [칠] 어 보완 실사 에서는 [12] 억원 중 영 [쩜] [팔] 학원 정도의 칠 소 가 발생 해 전체 출소일 길이 약 이 [쩜] [5] 억원으로 집계 도 였습니다 이러한 스치 는 시아 렘 시스템 과 고객 피드백 설문 결과 